In [99]:
import numpy as np
import pandas as pd
import sqlite3
import sys
import os
import re
from urllib.parse import urlparse

In [132]:
data_path = 'exp-data/'
file_name = "nyt-t1"
sql_file = data_path + file_name + '.sqlite'

In [133]:
query = """SELECT sv.site_url, sv.visit_id,
        hr.headers, hr.referrer, hr.url, hr.top_level_url
        FROM http_requests as hr LEFT JOIN site_visits as sv
        ON sv.visit_id = hr.visit_id
        """

In [134]:
conn = sqlite3.connect(sql_file)
# http_requests = pd.read_sql_query("select visit_id, headers, referrer, url, top_level_url from http_requests;", conn)
http_requests = pd.read_sql_query(query, conn)

In [135]:
cookies = pd.read_sql_query("select visit_id, record_type, change_cause, host, name, value from javascript_cookies;", conn)

In [136]:
def get_host_from_headers(x):
    temp = x.replace('"','').split('],[')
    for t in temp:
            pair = t.replace(']]','').replace('[[','').split(',')
            if "Host" in pair:
                host_value = pair[1]
    return host_value

def get_cookies_from_headers(x):
    ls = x.replace('"','').split('],[')
    for l in ls:
        ta = l.replace(']]','').split(',')
        if "Cookie" in ta:
            return ta[-1]
    return ''
def extract_host_from_url(url_ls):
    return list(map(lambda x: urlparse(x).netloc.split('.')[1] if len(urlparse(x).netloc.split('.')) > 1 else x, url_ls))

def get_site(site_url):
    return site_url.split(".")[1]
def is_third_party(origin, url_host):
    if origin in url_host:
        return False
    return True
def save_df_to_csv(df, file_name):
    df.to_csv(file_name+'.csv', index=False)

In [137]:
http_requests["host"] = list(map(lambda x: get_host_from_headers(x), http_requests.headers.tolist()))
http_requests["cookies"] =  list(map(lambda x: get_cookies_from_headers(x), http_requests.headers.tolist()))
http_requests["origin_site"] = list(map(lambda x: get_site(x), http_requests.site_url))
http_requests["is_third_party"] = list(map(lambda a, b: is_third_party(a, b), http_requests.origin_site, http_requests.host))

In [138]:
http_requests_third_party = http_requests[http_requests.is_third_party == True]


In [139]:
query2 = """SELECT sv.site_url, sv.visit_id,
        jsc.host, jsc.name, jsc.value
        FROM javascript_cookies as jsc LEFT JOIN site_visits as sv
        ON sv.visit_id = jsc.visit_id
        """
cookies = pd.read_sql_query(query2, conn)

In [140]:
# query2 = "select visit_id, record_type, change_cause, host, name, value, time_stamp from javascript_cookies;"
# 
cookies['origin_site'] = list(map(lambda x: get_site(x), cookies.site_url))
cookies['is_third_party'] = list(map(lambda a, b: is_third_party(a, b), cookies.origin_site, cookies.host))

In [141]:
cookies_first_party = cookies[cookies.is_third_party == False]
cookies_third_party = cookies[cookies.is_third_party == True]

In [142]:
# cookie_effective_values = [v.replace('|', '\|').replace("-", "\-").replace("[",'\[').replace(']','\]') for v in set(cookies.value) if len(v) > 5]
cookie_effective_values = [re.escape(v) for v in set(cookies_first_party.value) if len(v) > 5]
cookie_effective_all = [re.escape(v) for v in set(cookies.value) if len(v) > 5]
cookie_tp_effective = [re.escape(v) for v in set(cookies_third_party.value) if len(v) > 5]

In [143]:
fp_cookie_values = "|".join(cookie_effective_values)
all_cookie_values = "|".join(cookie_effective_all)
tp_cookie_values = "|".join(cookie_tp_effective)

In [161]:
len(tp_cookie_values)

45294

In [145]:
# df.loc[0:15,'A'] = 16

In [176]:
http_requests_third_party[http_requests_third_party.url.str.contains(fp_cookie_values, regex=True)]

,site_url,visit_id,headers,referrer,url,top_level_url,host,cookies,origin_site,is_third_party
38,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""dyv1bugovvq1g.cloudfront.net""],[""Use...",https://www.webmd.com/search/search_results/de...,https://dyv1bugovvq1g.cloudfront.net/25/www.we...,https://www.webmd.com/search/search_results/de...,dyv1bugovvq1g.cloudfront.net,,webmd,True
51,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""sb.scorecardresearch.com""],[""User-Ag...",https://www.webmd.com/search/search_results/de...,https://sb.scorecardresearch.com/b?c1=2&c2=603...,https://www.webmd.com/search/search_results/de...,sb.scorecardresearch.com,,webmd,True
53,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""s.tagsrvcs.com""],[""User-Agent"",""Mozi...",https://www.webmd.com/search/search_results/de...,https://s.tagsrvcs.com/2/587654/analytics.js?p...,https://www.webmd.com/search/search_results/de...,s.tagsrvcs.com,,webmd,True
69,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""sb.scorecardresearch.com""],[""User-Ag...",https://www.webmd.com/search/search_results/de...,https://sb.scorecardresearch.com/b2?c1=2&c2=60...,https://www.webmd.com/search/search_results/de...,sb.scorecardresearch.com,UID=16F18428a36a38a2808d1cc1606506090; UIDR=16...,webmd,True
72,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""mb.moatads.com""],[""User-Agent"",""Mozi...",https://www.webmd.com/search/search_results/de...,https://mb.moatads.com/yi.js?ud=1&qn=%604%7BZE...,https://www.webmd.com/search/search_results/de...,mb.moatads.com,,webmd,True
75,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""contextual.media.net""],[""User-Agent""...",https://www.webmd.com/search/search_results/de...,https://contextual.media.net/mcx.js?&rt=2&call...,https://www.webmd.com/search/search_results/de...,contextual.media.net,visitor-id=2495076903613114000V10,webmd,True
81,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""contextual.media.net""],[""User-Agent""...",https://www.webmd.com/search/search_results/de...,https://contextual.media.net/rtbsmpubs.php?&gd...,https://www.webmd.com/search/search_results/de...,contextual.media.net,visitor-id=2495076903613114000V10,webmd,True
82,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""contextual.media.net""],[""User-Agent""...",https://www.webmd.com/search/search_results/de...,https://contextual.media.net/rtbsmpubs.php?&gd...,https://www.webmd.com/search/search_results/de...,contextual.media.net,visitor-id=2495076903613114000V10,webmd,True
83,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""contextual.media.net""],[""User-Agent""...",https://www.webmd.com/search/search_results/de...,https://contextual.media.net/rtbsmpubs.php?&gd...,https://www.webmd.com/search/search_results/de...,contextual.media.net,visitor-id=2495076903613114000V10,webmd,True
91,https://www.webmd.com/search/search_results/de...,1,"[[""Host"",""px.moatads.com""],[""User-Agent"",""Mozi...",https://www.webmd.com/search/search_results/de...,https://px.moatads.com/pixel.gif?e=17&i=WEBMD_...,https://www.webmd.com/search/search_results/de...,px.moatads.com,,webmd,True


In [175]:
http_requests_syncs.url.str.contains(fp_cookie_values, regex=True).index

RangeIndex(start=0, stop=609, step=1)

In [169]:
http_requests_syncs['share_fp_cookie'] = [False]*http_requests_syncs.shape[0]
http_requests_syncs.loc[http_requests_syncs.url.str.contains(fp_cookie_values, regex=True).index, "share_fp_cookie"] = True
http_requests_syncs['share_tp_cookie'] = [False]*http_requests_syncs.shape[0]
http_requests_syncs.loc[http_requests_syncs.url.str.contains(tp_cookie_values, regex=True).index, "share_tp_cookie"] = True

In [170]:
http_requests_syncs[http_requests_syncs.share_fp_cookie==True].shape

(609, 16)

In [171]:
http_requests_syncs[http_requests_syncs.share_tp_cookie==True].shape

(609, 16)

In [151]:
# extract third party
http_requests_syncs['referrer_host'] = list(map(lambda x: urlparse(x).netloc, http_requests_syncs.referrer))

/Users/rainyleon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [152]:
# can tell wether referrer and url are different
http_requests_syncs['is_diff_host'] = list(map(lambda a, b: a != b, http_requests_syncs.host, http_requests_syncs.referrer_host))

/Users/rainyleon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [153]:
http_requests_syncs['referred_by_third'] = list(map(lambda a, b: is_third_party(a, b), http_requests_syncs.origin_site, http_requests_syncs.referrer_host))

/Users/rainyleon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [154]:
# cookie syncing referred by a different domain
http_requests_syncs[http_requests_syncs.is_diff_host].shape

(576, 15)

In [155]:
# only index 3
http_requests_syncs[http_requests_syncs.is_diff_host][http_requests_syncs.visit_id==3].shape

/Users/rainyleon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


(47, 15)

In [156]:
# cookie syncing between third parties 
http_requests_syncs[http_requests_syncs.is_diff_host][http_requests_syncs.referred_by_third].shape

/Users/rainyleon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


(223, 15)

In [157]:
# cookie not in header but in url
http_requests_syncs[http_requests_syncs.is_diff_host][http_requests_syncs.cookies==""].shape

/Users/rainyleon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


(235, 15)

In [55]:
save_df_to_csv(http_requests_syncs, "cookie_syncs_" +file_name)